# Grid Search - MLP


In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
import tensorflow as tf

from sklearn.utils import shuffle

df = pd.read_csv('african_crises.csv')
df = shuffle(df, random_state=11)
df.reset_index(inplace=True, drop=True)

df['banking_crisis'] = df['banking_crisis'].replace('crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(1)
df['banking_crisis'] = df['banking_crisis'].replace('no_crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(0)


df.drop(['cc3','country'], axis=1, inplace=True)


df_scaled = preprocessing.scale(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
df_scaled['banking_crisis'] = df['banking_crisis']
df = df_scaled

# defining the input data, X, and the desired results, y

X = df.loc[:,df.columns != 'banking_crisis']
y = df.loc[:, 'banking_crisis']



def create_model(learning_rate=0.01, hidden_units=16):
    # create model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(32, activation='relu', input_dim=11))
    model.add(tf.keras.layers.Dense(hidden_units, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='relu'))
    optimizer = tf.keras.optimizers.RMSprop(lr=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

template = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, verbose=0)

learning_rate = [0.01, 0.02, 0.05, 0.1]
batch_size = [100, 150, 200]
hidden_units = [8, 16, 32]
param_grid = dict(learning_rate=learning_rate, batch_size=batch_size, hidden_units=hidden_units)
gs = GridSearchCV(estimator=template, param_grid=param_grid, n_jobs=-1, cv=3)
gs_result = grid.fit(X, y)

#summarize results

means = gs_result.cv_results_['mean_test_score']
stds = gs_result.cv_results_['std_test_score']
params = gs_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("mean_acc: %f std_dev: (%f) hyper_parameters: %r" % (mean, stdev, param))
print("Best: %f using %s" % (gs_result.best_score_, gs_result.best_params_))

# Random Search - MLP


In [ ]:
from random import uniform

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import tensorflow as tf

from sklearn.utils import shuffle

df = pd.read_csv('african_crises.csv')
df = shuffle(df, random_state=11)
df.reset_index(inplace=True, drop=True)

df['banking_crisis'] = df['banking_crisis'].replace('crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(1)
df['banking_crisis'] = df['banking_crisis'].replace('no_crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(0)


df.drop(['cc3','country'], axis=1, inplace=True)


df_scaled = preprocessing.scale(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
df_scaled['banking_crisis'] = df['banking_crisis']
df = df_scaled


X = df.loc[:,df.columns != 'banking_crisis']
y = df.loc[:, 'banking_crisis']



def create_model(learning_rate=0.01, hidden_units=16):
    # create model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(32, activation='relu', input_dim=11))
    model.add(tf.keras.layers.Dense(hidden_units, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='relu'))
    optimizer = tf.keras.optimizers.RMSprop(lr=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, verbose=0)

# Define hyper_parameters values
learn_rate = [0.01, 0.02, 0.05, 0.1]
batch_size = [100, 150, 200]
hidden_units = [8, 16, 32]

n_iter_search = 10
param_grid = dict(learning_rate=learning_rate, batch_size=batch_size, hidden_units=hidden_units)

rs = RandomizedSearchCV(estimator=template, param_distributions=param_grid, n_iter=n_iter_search, n_jobs=-1, cv=3)
rs_result = rs.fit(X, y)

#summarize results

means = rs_result.cv_results_['mean_test_score']
stds = rs_result.cv_results_['std_test_score']
params = rs_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("mean_acc: %f std_dev: (%f) hyper_parameters: %r" % (mean, stdev, param))
print("Best: %f using %s" % (rs.best_score_, rs_result.best_params_))

# Sequential Model Bayesian Optimization - MLP


In [ ]:
from random import uniform

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.utils import shuffle
import sys

df = pd.read_csv('african_crises.csv')
df = shuffle(df, random_state=11)
df.reset_index(inplace=True, drop=True)

df['banking_crisis'] = df['banking_crisis'].replace('crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(1)
df['banking_crisis'] = df['banking_crisis'].replace('no_crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(0)


df.drop(['cc3','country'], axis=1, inplace=True)


df_scaled = preprocessing.scale(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
df_scaled['banking_crisis'] = df['banking_crisis']
df = df_scaled


X = df.loc[:,df.columns != 'banking_crisis']
y = df.loc[:, 'banking_crisis']


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2,shuffle=False)

space = {
            'hidden_units': hp.uniformint('hidden_units', 8,32),
            'batch_size': hp.uniformint('batch_size', 100,200),
            'learning_rate': hp.uniform('learning_rate', 0.01,0.1),
        }

#create model
def f_nn(params):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(32, activation='relu', input_dim=11))
    model.add(tf.keras.layers.Dense(params['hidden_units'], activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='relu'))
    optimizer = tf.keras.optimizers.RMSprop(lr=params['learning_rate'])
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


    model.fit(X_train, y_train, epochs=5, batch_size=params['batch_size'], verbose = 0)

    pred_auc =model.predict_proba(X_test, batch_size = params['batch_size'], verbose = 0)
    acc = roc_auc_score(y_test, pred_auc)
    print('Accuracy:', acc)
    sys.stdout.flush()
    return {'loss': -acc, 'status': STATUS_OK}

#search and output results 
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=50)
print('best: ')
print(best)